## import the library

In [ ]:
import slampy
import cv2
import numpy as np
from utils import *
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

%matplotlib notebook

## params
the configuration attribute of the algorithm can be found in the setting.yaml file,
you can change this file in order to execute different algorithm with different options

**image_folder** is the root folder of images 

In [ ]:
image_folder = 'Dataset/image_02'
setting_file ='settings.yaml'

initialize the system with the **setting_file** and the type of the sensor in this case monocular

In [ ]:
app = slampy.System(setting_file,slampy.Sensor.MONOCULAR)

## load the images
in this example we use the KITTI dataset, the images is not provided you can download it , in utils.py is also provide the method for read the TUM images named **load_images_TUM**

In [ ]:
image_filenames, timestamps = load_images_KITTI(image_folder)
num_images = len(image_filenames)

run the algorithm with the load dataset and plot the depth map found in each frame if the tracking is OK

In [ ]:
print('images in the sequences {}'.format(num_images))
fig = plt.figure()
ax = plt.subplot(111)
for idx in range(num_images):
    # load and convert to RGB image 
    name = image_filenames[idx]
    image = cv2.imread(name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    if image is None:
        print("failed to load image at {0}".format(self.idx))
        exit
    t1 = time.time()
    state = app.process_image_mono(image,timestamps[idx])
    t2 = time.time()
    if state == slampy.State.OK:
        #get the depth
        depth = app.get_depth()
        #plot the figure
        fig.suptitle('result of image {}'.format(idx))
        ax.imshow(depth,cmap='magma')
        fig.canvas.draw()
        
    #sleep the execution if the time is less than the image acquisition
    ttrack = t2 - t1
    t = 0
    if idx < num_images - 1:
        t = timestamps[idx + 1] - timestamps[idx]
    elif idx > 0:
        t = timestamps[idx] - timestamps[idx - 1]

    if ttrack < t:
        time.sleep(t - ttrack)
